In [1]:
import os
import pandas as pd
from kaggle.api.kaggle_api_extended import KaggleApi
from langchain.vectorstores import Chroma
from langchain.embeddings import SentenceTransformerEmbeddings
from sentence_transformers import SentenceTransformer
from transformers import pipeline

# Step 1: Authenticate with the Kaggle API
api = KaggleApi()
api.authenticate()

# Download the dataset (e.g., "All the News" dataset)
api.dataset_download_files('asad1m9a9h6mood/news-articles', path='data/', unzip=True)


c:\Users\neals\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Dataset URL: https://www.kaggle.com/datasets/asad1m9a9h6mood/news-articles


In [3]:

# Step 2: Load and Process the Dataset
file_path = 'data/Articles.csv'  # path based on dataset name
df = pd.read_csv(file_path, encoding='utf-8')
print(df.columns)

# Extract the content (text) column from the dataset
documents = df['content'].dropna().tolist()

# Convert documents into the required format for Chroma
document_list = [{"text": doc} for doc in documents[:100]]  # Limiting to 100 documents for demonstration


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xb4 in position 799: invalid start byte

In [ ]:

# Step 3: Embed Documents
embedder = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = SentenceTransformerEmbeddings(model_name='all-MiniLM-L6-v2')

# Create a vector store using Chroma
vector_store = Chroma.from_documents(document_list, embeddings)

# Step 4: Query the Vector Store
query = "Tell me about natural language processing"
retrieved_docs = vector_store.similarity_search(query, k=2)

# Step 5: Generate a Response Using Retrieved Documents as Context
context = " ".join([doc['text'] for doc in retrieved_docs])

# Use an LLM for response generation
llm = pipeline("text-generation", model="gpt2")  # Replace with another model if needed
response = llm(context, max_length=150, do_sample=True, top_k=50)[0]['generated_text']

print(f"Generated Response: {response}")

